# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [1]:
from typing import Tuple, List

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove
from copy import copy
import numpy as np
from math import inf



Wielkość planszy

In [2]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [3]:
class Max(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

class Min(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

In [249]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        if type(game.first_player) is not Max and type(game.first_player) is not Min:
            raise ValueError("The first player is not Max nor Min")
        if type(game.second_player) is not Max and type(game.second_player) is not Min:
            raise ValueError("The second player is not Max nor Min")
        if type(game.first_player) is type(game.second_player):
            raise ValueError("Both players are the same type; One should be Max and one should be Min")
        self.game = game
        self.column_count = len(self.game.state.fields)
        self.row_count = len(self.game.state.fields[0])
        

    def evaluate_position(self, player: Player)->float:
        if self.game.is_finished():
            winner = self.game.get_winner()
            if type(winner) is Max:
                return inf
            elif type(winner) is Min:
                return -inf
            else:
                return 0.0
        else:
            evaluation = self._evaluate_score(player)
            return evaluation

    def _evaluate_score(self, player: Player) -> float:
        opponent = self.game.get_players()[0]
        if opponent is player:
            opponent = self.game.get_players()[1]
        score = 0.0
        score += self._check_columns(player, opponent)
        score += self._check_rows(player, opponent)
        score += self._check_diagonals(player, opponent)

    
        if type(player) is Min:
            score = -score
        return score
    
    def _check_columns(self, player: Player, opponent: Player) -> float:
        score = 0.0
        for column in range(self.column_count):
            for starting_row in range(self.row_count - 3):
                four_fields = self.game.state.fields[column][starting_row:starting_row+4]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
        return score
    
    def _check_rows(self, player: Player, opponent: Player) -> float:
        score = 0.0
        for row in range(self.row_count):
            for starting_column in range(self.column_count - 3):
                four_fields = [self.game.state.fields[starting_column + i][row] for i in range(4)]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
        return score
    
    def _check_diagonals(self, player: Player, opponent: Player) -> float:
        score = 0.0
        for starting_column in range(self.column_count - 3):
            for starting_row in range(self.row_count - 3):
                four_fields = [self.game.state.fields[starting_column + i][starting_row + i] for i in range(4)]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
                four_fields = [self.game.state.fields[starting_column + i][self.row_count - 1 - starting_row - i] for i in range(4)]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
        return score
    
    def _score_four_fields(self, player: Player, four_fields: list) -> float:
        if four_fields.count(player) + four_fields.count(None) == 4:
            return 1
        return 0

    def is_valid_move(self, col_index:int)->bool:
        return self.game.state.fields[col_index][ROW_COUNT - 1] is None
    
    def get_valid_moves(self) -> list:
        valid_moves = []
        for column in range(COLUMN_COUNT):
            if self.is_valid_move(column):
                valid_moves.append(column)
        return valid_moves

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        valid_moves = self.get_valid_moves()
        if self.game.is_finished() or depth == 0:
            player = self.game.get_players()[1] #player that makes the last move in the evaluated postion
            return None, self.evaluate_position(player)
        else:
            best_move = valid_moves[np.random.choice(len(valid_moves))]
            if is_maximizing_player:
                evaluation = -inf
                for move in valid_moves:
                    new_evaluation = self._evaluate_next_state(move, depth, alpha, beta, is_maximizing_player)
                    if new_evaluation > evaluation:
                        evaluation = new_evaluation
                        best_move = move
                    alpha = max(alpha, evaluation)
                    if alpha >= beta:
                        break
                return best_move, alpha
            else:
                evaluation = inf
                for move in valid_moves:
                    new_evaluation = self._evaluate_next_state(move, depth, alpha, beta, is_maximizing_player)
                    if new_evaluation < evaluation:
                        evaluation = new_evaluation
                        best_move = move
                    beta = min(beta, evaluation)
                    if alpha >= beta:
                        break
                return best_move, beta
    def _evaluate_next_state(self, move: int, depth: int, alpha:float, beta: float, is_maximizing_player: bool) -> float:
        evaluated_game = copy(self.game)
        evaluated_game.make_move(ConnectFourMove(move))
        new_solver = MinMaxSolver(evaluated_game)
        return new_solver.minimax(depth-1, alpha, beta, not is_maximizing_player)[1]    
            


Rozgrywka

In [215]:

player_one = Max("x")
player_two = Min("o")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=player_one, second_player=player_two)
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))

s = MinMaxSolver(game)
print(s.minimax(3, -inf, inf, True))



(6, inf)


In [223]:
def play_game(depth_one: int, depth_two: int):
    player_one = Max("a")
    player_two = Min("b")
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=player_one, second_player=player_two)
    solver = MinMaxSolver(game)
    while not game.is_finished():
        move, evaluation = solver.minimax(depth_one, -inf, inf, type(solver.game.get_current_player()) is Max)
        solver.game.make_move(ConnectFourMove(move))
        print(f"Player one move: {move}, evaluation {evaluation}")
        print(game.state)
        if game.get_winner():
            break
        move, evaluation = solver.minimax(depth_two, -inf, inf, type(solver.game.get_current_player()) is Max)
        solver.game.make_move(ConnectFourMove(move))
        print(f"Player two move: {move}, evaluation: {evaluation}")
        print(game.state)
    


In [220]:
play_game(6, 4)

Player one move: 3, evaluation -1.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 12.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 13.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -7.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 16.0
Current player: a
[ ][ ][ ][b][

In [224]:
play_game(7, 5)

Player one move: 3, evaluation 5.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 2, evaluation 3.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][ ][ ][ ]
Player two move: 1, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][b][a][a][ ][ ][ ]
Player one move: 3, evaluation 3.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][b][a][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][

In [235]:
play_game(8, 5)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 1, evaluation -3.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][a][ ][a][ ][ ][ ]
Player two move: 2, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][a][b][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][a][b][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][

In [250]:
play_game(4, 4)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 6.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 3.0
Current player: a
[ ][ ][ ][b][ ][

In [226]:
play_game(5,5)

Player one move: 1, evaluation 6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player two move: 1, evaluation: -4.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player one move: 1, evaluation 6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player two move: 1, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player one move: 3, evaluation 5.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][a][ ][ ][ ]
Player two move: 4, evaluation: -1.0
Current player: a
[ ][ ][ ][ ][ ][

In [227]:
play_game(4, 5)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][b][

In [228]:
play_game(4, 6)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 5.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 2.0
Current player: a
[ ][ ][ ][b][ ][

In [229]:
play_game(4, 7)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][b][

In [234]:
play_game(1, 1)

Player one move: 3, evaluation 7.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation 9.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation 6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][b][ ][

In [236]:
play_game(4, 1)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][b][

In [238]:
play_game(6, 6)

Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 5.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 2, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][ ][ ][ ]
Player two move: 4, evaluation: 3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][b][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][b][ ][ ]
Player two move: 3, evaluation: 2.0
Current player: a
[ ][ ][ ][ ][ ][

In [246]:
def play_with_computer(depth: int):
    player_one = Max("a")
    player_two = Min("b")
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=player_one, second_player=player_two)
    solver = MinMaxSolver(game)
    while not game.is_finished():
        print(game.state)
        move = int(input("Enter your move: "))
        if solver.is_valid_move(move):
            solver.game.make_move(ConnectFourMove(move))
        else:
            continue
        evaluation = solver.evaluate_position(player_one)
        print(f"Player one move: {move}, evaluation {evaluation}")
        if game.get_winner():
            break
        print(game.state)
        move, evaluation = solver.minimax(depth, -inf, inf, type(solver.game.get_current_player()) is Max)
        solver.game.make_move(ConnectFourMove(move))
        print(f"Player two move: {move}, evaluation: {evaluation}")
    print(game.state)
        

In [248]:
play_with_computer(5)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Player one move: 3, evaluation 7.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation 9.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation 6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ]